In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cmocean as cmo
from matplotlib.patches import Polygon
from copy import copy
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [2]:
lw_voronoi = 0.
lw_gl = .2
ec_voronoi = None #'w'

plotannual = True

In [3]:
#    C%type_icefree_land                        = 1
#    C%type_icefree_ocean                       = 2
#    C%type_grounded_ice                        = 3
#    C%type_floating_ice                        = 4
#    C%type_groundingline_gr                    = 5
#    C%type_groundingline_fl                    = 6
#    C%type_calvingfront_gr                     = 7
#    C%type_calvingfront_fl                     = 8
#    C%type_margin                              = 9
#    C%type_coastline                           = 10

In [4]:
vmax = 100
vmin = -10
linthresh = .3
linscale = .25

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
nneg = np.int_((1-fracpos)*256)

#colors1 = plt.get_cmap('cmo.dense_r')(np.linspace(0,1.,nneg+1))
#colors2 = plt.get_cmap('gist_heat_r')(np.linspace(0., 1, 256-nneg-1))

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,1.,nneg+1))
colors2 = plt.get_cmap('inferno')(np.linspace(0., 1, 256-nneg-1))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)
norm1=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale)

cmap2 = plt.get_cmap('gist_stern')
norm2 = mpl.colors.Normalize(vmin=0,vmax=800)

cmap3 = plt.get_cmap('cmo.rain')
norm3 = mpl.colors.Normalize(vmin=0,vmax=1)

cmap4 = plt.get_cmap('tab10')
norm4 = mpl.colors.Normalize(vmin=0.5,vmax=10.5)

cmap5 = plt.get_cmap('turbo')
norm5 = mpl.colors.Normalize(vmin=0,vmax=2000)

cmap6 = plt.get_cmap('cmo.speed')
norm6 = mpl.colors.Normalize(vmin=0,vmax=.6)

cmap7 = plt.get_cmap('cmo.rain')
norm7 = mpl.colors.Normalize(vmin=0,vmax=100)

ticks1 = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
ticks2 = [0,200,400,600,800]
ticks4 = np.arange(1,11)

In [ ]:
t = 0
f = 0

for s in [1]: #range(1,25):

    #Create figure
    fig = plt.figure(figsize=(15,10))

    gs = fig.add_gridspec(3,1, height_ratios=(1, 1, 1),left=0.05, right=0.99, bottom=0.05, top=0.95, wspace=0.1, hspace=0.02)

    ax0 = fig.add_subplot(gs[0])
    ax1 = fig.add_subplot(gs[1])
    ax2 = fig.add_subplot(gs[2])

    #Add colorbars
    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap=cmap1),ax=ax0,extend='both',shrink=.8)
    cb.set_ticks(ticks1)
    cb.set_ticklabels(ticks1)
    cb.set_label("Melt rate [m.i.e./yr]")

    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm6, cmap=cmap6),ax=ax1,extend='max',shrink=.8)
    #cb.set_ticks(ticks2)
    #cb.set_ticklabels(ticks2)
    cb.set_label("Ocean speed [m/s]")

    #cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm4, cmap=cmap4),ax=ax1,shrink=.8)
    #cb.set_ticks(ticks4)
    #cb.set_ticklabels(ticks4)
    #cb.set_label("Mask")

    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm5, cmap=cmap5),ax=ax2,extend='max',shrink=.8)
    #cb.set_ticks(ticks5)
    #cb.set_ticklabels(ticks5)
    cb.set_label("Ice speed [m/yr]")

    for Ax in [ax0,ax1]:
        Ax.set_xticklabels([])
        #Ax.set_yticklabels([])

    #Open data on new mesh
    #ds = xr.open_dataset(f'../output/MISMIPplus_2km_runtime/temp.nc')
    #ds = xr.open_dataset(f'../output/MISOMIP_8km_spinup/main_output_ANT_{s:05d}.nc')
    ds = xr.open_dataset(f'../output/MISMIP_run1e/main_output_ANT_{s:05d}.nc')

    #Make up axes and labels
    for Ax in [ax0,ax1,ax2]:
        Ax.set_xlim(3.5e5,6.5e5)
        Ax.set_ylim(ds.ymin,ds.ymax)
        Ax.set_aspect(1)
        Ax.grid(color='.3',linewidth=.1)

    #Allocate for storage of polygons
    BMB = {}
    VAR1 = {}
    VAR2 = {}

    # Gather points surrounding vertex (quasi-voronoi)
    for v in range(len(ds.vi)):
        polyx = []
        polyy = []
        for n in range(ds.niTri[v].data):
            c = ds.iTri[n,v].data-1 #Neighbouring triangle
            polyx.append(ds.Tricc[0,c].data)
            polyy.append(ds.Tricc[1,c].data)             
        #TO DO: Append edge points here

        #Accumulate into polygons
        points = np.array([polyx,polyy]).T #Converted to km
        BMB[v] = Polygon(points,edgecolor=ec_voronoi,linewidth=lw_voronoi)
        VAR2[v]  = Polygon(points,edgecolor=ec_voronoi,linewidth=lw_voronoi)

        #Add polygons to axes
        ax0.add_patch(BMB[v])
        ax2.add_patch(VAR2[v])
    
    for ti in range(len(ds.ti)):
        polyx = []
        polyy = []
        for n in range(3):
            c = ds.Tri[n,ti].data-1 #Neighbouring vertex
            polyx.append(ds.V[0,c].data)
            polyy.append(ds.V[1,c].data)             

        #Accumulate into polygons
        points = np.array([polyx,polyy]).T #Converted to km
        VAR1[ti]  = Polygon(points,edgecolor=ec_voronoi,linewidth=lw_voronoi)

        ax1.add_patch(VAR1[ti])

    print(f"{s:02d}: Added cells")

    #Update colors for each time step within this mesh
    print(f"{s:02d}: {len(ds.time)} time slices")
    for i in range(len(ds.time)):
        
        #Plot only annual values; continue if not whole year
        #if (plotannual and t>np.floor(t)):
        #    t += 0.25
        #    continue

        # Loop over triangles
        for ti in range(len(ds.ti)):
            uabs = (ds.U_lad[i,ti]**2+ds.V_lad[i,ti]**2)**.5
            if uabs > 0:
                VAR1[ti].set_facecolor(cmap6(norm6((ds.U_lad[i,ti]**2+ds.V_lad[i,ti]**2)**.5)))
            elif ds.fraction_gr_b[i,ti]==0:
                VAR1[ti].set_facecolor('teal')
            else:
                VAR1[ti].set_facecolor('.8')

        #Loop over vertex to fill colours and plot grounding line
        for v in range(len(ds.vi)):

            #Hi[v].set_facecolor(cmap4(norm4(ds.mask[i,v])))
            
            #Fill colours based on data
            if ds.mask[i,v].data in [2]: #Ocean
                BMB[v].set_facecolor('teal')
                VAR2[v].set_facecolor('teal')
            elif ds.mask[i,v].data in [4,6,8]: #Ice shelf
                BMB[v].set_facecolor(cmap1(norm1(-ds.BMB[i,v])))
                VAR2[v].set_facecolor(cmap5(norm5(ds.uabs_vav[i,v])))
            elif (ds.mask[i,v].data in [5]): # and ds.BMB[i,v] != 0): #gl_gr
                BMB[v].set_facecolor('.8')#cmap1(norm1(-ds.BMB[i,v]))) #Grounded with possible BMB from PMP
                VAR2[v].set_facecolor(cmap5(norm5(ds.uabs_vav[i,v])))
            else: #Grouded
                BMB[v].set_facecolor('.8')
                VAR2[v].set_facecolor(cmap5(norm5(ds.uabs_vav[i,v])))
            
            # Plot grounding line segments
            if ds.mask[i,v].data in [5,7]: #gl_gr or ca_gr
                for n in range(ds.nC[v].data):
                    c = ds.C[n,v].data-1
                    if ds.mask[i,c].data in [6,8]: #neighbouring gl_fl or ca_fl
                        cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                        for Ax in [ax0,ax1,ax2]:
                            Ax.plot(ds.Tricc[0,cs]/1000,ds.Tricc[1,cs],lw=lw_gl,c='yellow')
                    elif ds.mask[i,c].data in [2]: #Ocean
                        cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                        for Ax in [ax0,ax1,ax2]:
                            Ax.plot(ds.Tricc[0,cs]/1000,ds.Tricc[1,cs],lw=lw_gl,c='orange')                       

            # Plot grounding line segments
            if ds.mask[i,v].data in [5,7]: #gl_gr or ca_gr
                for n in range(ds.nC[v].data):
                    c = ds.C[n,v].data-1
                    if ds.mask[i,c].data in [6,8]: #neighbouring gl_fl or ca_fl
                        cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                        lenn = .5*(ds.R[v]+ds.R[c])
                        for Ax in [ax0,ax1,ax2]:
                            Ax.plot(ds.Tricc[0,cs],ds.Tricc[1,cs],lw=lw_gl*lenn/1000,c='yellow',solid_capstyle='round')
                    elif ds.mask[i,c].data in [2]: #Ocean
                        cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                        lenn = .5*(ds.R[v]+ds.R[c])
                        for Ax in [ax0,ax1,ax2]:
                            Ax.plot(ds.Tricc[0,cs],ds.Tricc[1,cs],lw=lw_gl*lenn/1000,c='orange',solid_capstyle='round')  

        ax0.set_title(f"Year {t:10.2f}")

        #Save figure
        #canvas = FigureCanvas(fig)
        #canvas.print_figure(f'../video/frame_{f:03d}.png',dpi=1200)
        plt.savefig(f'../video2/frame_{f:03d}.png',dpi=600)
        #plt.savefig(f'../video2/frame_{f:03d}.pdf')
        
        #Remove grounding line
        for Ax in [ax0,ax1,ax2]:
            for line in Ax.get_lines():
                line.remove()

        print(f"{s:02d}: Made frame {f:03d} (time: {t:10.2f})")

        #Increase counters
        t += 1
        f += 1

    #Close data set to prepare for new mesh
    ds.close()

    plt.close()

    print(f"{s:02d}: All clear")
print(f"t = {t}")
print(f"f = {f}")

01: Added cells
01: 51 time slices
01: Made frame 000 (time:       0.00)
01: Made frame 001 (time:       1.00)
01: Made frame 002 (time:       2.00)
01: Made frame 003 (time:       3.00)
01: Made frame 004 (time:       4.00)
01: Made frame 005 (time:       5.00)
01: Made frame 006 (time:       6.00)
01: Made frame 007 (time:       7.00)
01: Made frame 008 (time:       8.00)
01: Made frame 009 (time:       9.00)
01: Made frame 010 (time:      10.00)
01: Made frame 011 (time:      11.00)
01: Made frame 012 (time:      12.00)
01: Made frame 013 (time:      13.00)
01: Made frame 014 (time:      14.00)
01: Made frame 015 (time:      15.00)
01: Made frame 016 (time:      16.00)
01: Made frame 017 (time:      17.00)
01: Made frame 018 (time:      18.00)
01: Made frame 019 (time:      19.00)


KeyboardInterrupt: 

In [5]:
#Make video
moviename = 'mismip_tanh_high_Kh10'
if plotannual:
    framerate = 10
else:
    framerate = 40

command = f'ffmpeg -r {framerate} -f image2 -i ../video2/frame_%03d.png -pix_fmt yuv420p -vcodec libx264 -crf 24 ../video2/{moviename}.mp4'
print(command)
#os.system(command)

#os.system('rm -r ../video/frame_*.png')

ffmpeg -r 10 -f image2 -i ../video2/frame_%03d.png -pix_fmt yuv420p -vcodec libx264 -crf 24 ../video2/mismip_tanh_high_Kh10.mp4


In [ ]:
ds